In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        
        # 输入图像channel：1；
        # 输出图像channel：6；5x5卷积核
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # 2x2 Max pooling
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # 如果是方阵,则可以只使用一个数字进行定义
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # 除去批处理维度的其他所有维度
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [3]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [4]:
params = list(net.parameters())
print(len(params))

10


In [5]:
for i in range(len(params)):
    print(params[i].size())    # layer's weight

torch.Size([6, 1, 5, 5])
torch.Size([6])
torch.Size([16, 6, 5, 5])
torch.Size([16])
torch.Size([120, 400])
torch.Size([120])
torch.Size([84, 120])
torch.Size([84])
torch.Size([10, 84])
torch.Size([10])


In [6]:
input = torch.randn(3, 1, 32, 32)  # 3, batch 
out = net(input)
print(out)

tensor([[ 0.0267, -0.0870,  0.0295,  0.0008,  0.0428, -0.0135,  0.0643, -0.0397,
          0.0086, -0.0965],
        [ 0.0442, -0.0609,  0.0211, -0.0051,  0.0427, -0.0071,  0.0448, -0.0159,
          0.0298, -0.1210],
        [ 0.0273, -0.0769,  0.0227, -0.0164,  0.0541, -0.0132,  0.0408, -0.0293,
          0.0124, -0.1220]], grad_fn=<AddmmBackward>)


In [7]:
print( net.num_flat_features(input) )
net.zero_grad()

1024


In [8]:
out.backward(torch.randn(3, 10), retain_graph=True)

In [9]:
target = torch.randn(3, 10)  # 本例子中使用模拟数据
target = target.view(3, -1)  # 使目标值与数据值形状一致

criterion = nn.MSELoss()

loss = criterion(out, target)
print(loss)

tensor(0.5369, grad_fn=<MseLossBackward>)


In [10]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

In [11]:
net.zero_grad()     # 清零所有参数（parameter）的梯度缓存

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward(retain_graph=True)

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0032,  0.0030, -0.0004, -0.0039, -0.0047,  0.0093])


In [12]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [13]:
import torch.optim as optim

# 创建优化器（optimizer）
optimizer = optim.SGD(net.parameters(), lr=0.01)

# 在训练的迭代中：
optimizer.zero_grad()   # 清零梯度缓存
output = net(input)

loss = criterion(output, target)
loss.backward(retain_graph=True)
optimizer.step()    # 更新参数

In [14]:
optimizer

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.01
    momentum: 0
    nesterov: False
    weight_decay: 0
)